In [ ]:
!pip install ctgan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from ctgan import CTGAN
from sklearn.preprocessing import MinMaxScaler

## Why CTGAN for synthetic data generation?

CTGAN is suitable for our synthetic data generation because it can effectively manage mixed data types (binary, categorical, and continuous) present in our dataset. Its ability to capture complex relationships between variables ensures that the synthetic data maintains the statistical properties of the original records. Additionally, CTGAN allows for conditional generation, which can help create tailored samples based on specific features, enhancing the relevance and realism of the synthetic data we produce.

In [ ]:
#contains all 36 metrics as well as the soldier name which is kept soldier_0, soldier_1 etc since we do not know their language and how'd they would name their people.
def create_empty_betrayal_metrics_df():
    columns = [
        'username','name','superhero_name', "preffered_age_range","sexuality","gender","city","education_level","income_level","curiosity_level","organized_chaos","social_butterfly","team_player_vibes","chill_factor","adventure_seeker","perfectionist_mode","party_starter","harmony_seeker","mood_meter","dietary_preferences","pet_preferences","fitness_preferences","smoking_habits","alcohol_consumption","preferred_occupation1","preferred_occupation2","preferred_occupation3","education_level","income_level","hobby1","hobby2","hobby3","movie_preference1","movie_preference2","movie_preference3","song_preference1","song_preference2","song_preference3","have_dated","dating_status"
    ]
    df = pd.DataFrame(columns=columns)
    return df

In [ ]:
database_df = create_empty_betrayal_metrics_df()

In [ ]:

binary_columns = [
    "gender","smoking_habits","dating_status","have_dated"
]

# continuous_columns = [
#     'risk', 'discipline','edu_level','mental_health','risk_last_op','active_duty_record','failure'
# ]

integer_columns = list(set(database_df.columns) - set(binary_columns)  - {"username"} - {"name"}- {"superhero_name"} - {"city"})

In [ ]:
variable_ranges = {
    # # Continuous variables with 0-1 range
    # 'risk': (0, 1),
    # 'risk_last_op': (0, 1),
    # 'mental_health': (0, 1),
    # 'edu_level': (0, 1),
    # 'discipline': (0, 1),
    # 'active_duty_record': (0, 1),
    # 'failure': (0, 1),

    # Integer variables with specific ranges
    'preffered_age_range': (0,6),
    'sexuality': (0,4),
    'education_level':(0,4),
    'income_level':(0,4),
    'curiosity_level':(0,10),
    'organized_chaos':(0,10),
    'social_butterfly':(0,10),
    'team_player_vibes':(0,10),
    'chill_factor':(0,10),
    'adventure_seeker':(0,10),
    'perfectionist_mode':(0,10),
    'party_starter':(0,10),
    'harmony_seeker':(0,10),
    'mood_meter':(0,10),
    'dietary_preferences':(0,4),
    'pet_preferences':(0,3),
    'fitness_preferences':(0,4),
    'alcohol_consumption':(0,3),
    'preferred_occupation1':(0,6),
    'preferred_occupation2':(0,6),
    'preferred_occupation3':(0,6),
    'hobby1':(0,7),
    'hobby2':(0,7),
    'hobby3':(0,7),
    'movie_preference1':(0,8),
    'movie_preference2':(0,8),
    'movie_preference3':(0,8),
    'song_preference1':(0,8),
    'song_preference2':(0,8),
    'song_preference3':(0,8),
}


In [ ]:
def generate_random_data(n_samples=50):
    data = []
    for _ in range(n_samples):
        sample = {}

        for col in binary_columns:
            sample[col] = np.random.choice([0, 1])

        for col in integer_columns:
            sample[col] = np.random.randint(variable_ranges[col][0], variable_ranges[col][1]+1)

        sample['name'] = 'placeholder_name'

        data.append(sample)

    return pd.DataFrame(data)

In [ ]:
initial_data = generate_random_data(n_samples=500)

df_train = initial_data.drop(columns=['name'])

In [ ]:
#normalising the data
# scaler = MinMaxScaler()
# df_train_normalized = pd.DataFrame(scaler.fit_transform(df_train), columns=df_train.columns)

In [ ]:
ctgan = CTGAN(epochs=1000)
ctgan.fit(df_train)

In [ ]:
synthetic_data_normalized = ctgan.sample(500)

In [ ]:
synthetic_data_normalized

,gender,smoking_habits,dating_status,have_dated,hobby2,sexuality,movie_preference2,hobby3,party_starter,preffered_age_range,...,chill_factor,income_level,pet_preferences,mood_meter,fitness_preferences,harmony_seeker,alcohol_consumption,song_preference1,dietary_preferences,song_preference3
0,1,1,1,1,9,1,8,2,4,1,...,7,1,1,3,3,7,1,7,2,8
1,0,0,0,1,1,2,1,0,5,1,...,6,3,0,3,2,-1,3,3,2,5
2,0,0,0,1,5,0,2,2,9,1,...,13,0,3,10,3,4,2,2,3,7
3,1,0,0,1,3,1,6,2,5,4,...,7,0,2,9,0,2,2,6,3,9
4,1,1,0,1,4,0,6,4,4,3,...,9,1,0,4,3,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,0,0,0,1,4,2,8,6,4,1,...,8,0,1,10,2,4,2,5,1,7
496,1,0,0,0,1,4,1,5,8,4,...,5,2,3,8,-1,0,2,6,3,9
497,1,0,1,0,4,4,0,1,0,0,...,4,1,1,2,2,3,1,8,0,7
498,0,1,0,1,3,3,8,6,5,6,...,9,3,2,13,3,4,1,2,2,7


Denormalizing in CTGAN is essential for the following reasons since

synthetic data generated by CTGAN is often normalized to improve training stability. Denormalizing the data restores it to its original scale, making it easier to interpret and use in practical applications

Ensures that the generated data accurately reflects the actual value ranges and distributions of the original dataset, enhancing its realism and usability.


a common challenge when using generative models like CTGAN is that they learn the overall distribution of the data but don't inherently understand hard constraints like minimum and maximum values.

so we apply post-processing to the data

In [ ]:
synthetic_data_normalized.describe()

,gender,smoking_habits,dating_status,have_dated,hobby2,sexuality,movie_preference2,hobby3,party_starter,preffered_age_range,...,chill_factor,income_level,pet_preferences,mood_meter,fitness_preferences,harmony_seeker,alcohol_consumption,song_preference1,dietary_preferences,song_preference3
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,...,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.430000,0.516000,0.358000,0.426000,3.426000,1.702000,4.820000,3.558000,4.518000,3.064000,...,7.420000,1.650000,1.616000,8.108000,1.900000,4.700000,1.474000,4.698000,1.666000,5.132000
std,0.495572,0.500244,0.479892,0.494989,2.588034,1.347898,3.051922,2.248722,3.136455,2.169174,...,3.410405,1.544108,1.134512,3.433353,1.383412,3.413694,1.040909,2.838117,1.419321,2.681222
min,0.000000,0.000000,0.000000,0.000000,-2.000000,-1.000000,-2.000000,-2.000000,-2.000000,-1.000000,...,-1.000000,-1.000000,0.000000,0.000000,-1.000000,-2.000000,0.000000,-2.000000,-1.000000,-1.000000
25%,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,2.000000,2.000000,1.000000,...,4.000000,0.000000,1.000000,5.000000,1.000000,2.000000,1.000000,2.000000,1.000000,3.000000
50%,0.000000,1.000000,0.000000,0.000000,3.000000,1.000000,5.500000,4.000000,4.000000,3.000000,...,8.000000,1.000000,1.000000,9.000000,2.000000,5.000000,1.000000,5.000000,2.000000,5.000000
75%,1.000000,1.000000,1.000000,1.000000,6.000000,3.000000,7.000000,5.000000,6.000000,5.000000,...,10.000000,3.000000,3.000000,11.000000,3.000000,7.250000,2.000000,7.000000,3.000000,7.000000
max,1.000000,1.000000,1.000000,1.000000,9.000000,5.000000,10.000000,9.000000,13.000000,7.000000,...,13.000000,5.000000,4.000000,13.000000,4.000000,12.000000,4.000000,10.000000,5.000000,10.000000


In [ ]:
def clip_to_range(data, ranges):
    for col, (min_val, max_val) in ranges.items():
        if col in data.columns:
            data[col] = data[col].clip(min_val, max_val)
    return data

for col in binary_columns:
    synthetic_data_normalized[col] = synthetic_data_normalized[col].round().astype(int)

synthetic_data_normalized = clip_to_range(synthetic_data_normalized, variable_ranges)

for col in integer_columns:
    if col in synthetic_data_normalized.columns:
        synthetic_data_normalized[col] = synthetic_data_normalized[col].round().astype(int)


In [ ]:
synthetic_data_normalized['name'] = ['user_' + str(i) for i in range(len(synthetic_data_normalized))]

In [ ]:
#verification of adherence to ranges
for col, (min_val, max_val) in variable_ranges.items():
    if col in synthetic_data_normalized.columns:
        actual_min = synthetic_data_normalized[col].min()
        actual_max = synthetic_data_normalized[col].max()
        if actual_min < min_val or actual_max > max_val:
            print(f"Warning: {col} has values outside the specified range. Min: {actual_min}, Max: {actual_max}")

In [ ]:
!pip install skimpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 50.8 MB/s eta 0:00:00
  Attempting uninstall: typeguard
    Found existing installation: typeguard 4.3.0
    Uninstalling typeguard-4.3.0:
      Successfully uninstalled typeguard-4.3.0
  Attempting uninstall: polars
    Found existing installation: polars 1.7.1
    Uninstalling polars-1.7.1:
      Successfully uninstalled polars-1.7.1
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 5.5.6
    Uninstalling ipykernel-5.5.6:
      Successfully uninstalled ipykernel-5.5.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipykernel==5.5.6, but you have ipykernel 6.29.5 which is incompatible.


In [ ]:
import skimpy as skim

skim.skim(synthetic_data_normalized)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 500    │ │ int64       │ 34    │                                                          │
│ │ Number of columns │ 35     │ │ string      │ 1     │                                                          │
│ └───────────────────┴────────┘ └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┳━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━┳━━━━━━┳━━━━━━━━┓  │
│ ┃ column_name               ┃ NA  ┃ NA %   ┃ mean    ┃ sd       ┃ p0  ┃ p25   ┃ p50  ┃ p75   ┃ p100 ┃ hist   ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━╇━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━╇━━━━━━╇━━━━━━━━┩  │
│ │ gender                    │   0 │      0 │    0.43 │   0.4956 │   0 │     0 │    0 │     1 │    1 │ ▇    ▆ │  │
│ │ smoking_habits            │   0 │      0 │   0.516 │   0.5002 │   0 │     0 │    1 │     1 │    1 │ ▇    ▇ │  │
│ │ dating_status             │   0 │      0 │   0.358 │   0.4799 │   0 │     0 │    0 │     1 │    1 │ ▇    ▅ │  │
│ │ have_dated                │   0 │      0 │   0.426 │    0.495 │   0 │     0 │    0 │     1 │    1 │ ▇    ▆ │  │
│ │ hobby2                    │   0 │      0 │   3.398 │    2.386 │   0 │     1 │    3 │     6 │    7 │ ▇▅▃▂▃▇ │  │
│ │ sexuality                 │   0 │      0 │   1.678 │    1.289 │   0 │     1 │    1 │     3 │    4 │ ▅▇ ▅▃▃ │  │
│ │ movie_preference2         │   0 │      0 │   4.752 │    2.787 │   0 │     2 │  5.5 │     7 │    8 │ ▅▂▁▃▃▇ │  │
│ │ hobby3                    │   0 │      0 │   3.572 │    2.133 │   0 │     2 │    4 │     5 │    7 │ ▇▆▅▅▅▇ │  │
│ │ party_starter             │   0 │      0 │    4.51 │    2.947 │   0 │     2 │    4 │     6 │   10 │ ▅▇▅▇▃▅ │  │
│ │ preffered_age_range       │   0 │      0 │   3.018 │    2.019 │   0 │     1 │    3 │     5 │    6 │ ▃▆▅▃▃▇ │  │
│ │ movie_preference3         │   0 │      0 │   3.908 │    2.704 │   0 │     1 │    4 │     6 │    8 │ ▇▂▂▇▅▅ │  │
│ │ song_preference2          │   0 │      0 │   4.414 │    2.384 │   0 │  2.75 │    5 │     6 │    8 │ ▅▃▃▇▃▇ │  │
│ │ adventure_seeker          │   0 │      0 │   6.052 │    3.398 │   0 │     3 │    7 │    10 │   10 │ ▂▅▂▂▃▇ │  │
│ │ movie_preference1         │   0 │      0 │   5.106 │    2.572 │   0 │     3 │    6 │     8 │    8 │ ▂▂▃▃▂▇ │  │
│ │ hobby1                    │   0 │      0 │    3.94 │    2.534 │   0 │     2 │    4 │     6 │    7 │ ▅▃▁▂▃▇ │  │
│ │ perfectionist_mode        │   0 │      0 │   3.624 │    3.197 │   0 │     0 │    3 │     6 │   10 │ ▇▃▂▃▃▂ │  │
│ │ preferred_occupation1     │   0 │      0 │    3.08 │    1.716 │   0 │     2 │    3 │     4 │    6 │ ▁▃▇▅▅▇ │  │
│ │ team_player_vibes         │   0 │      0 │   3.976 │    3.347 │   0 │     1 │    3 │     7 │   10 │ ▇▅▂▃▅▃ │  │
│ │ education_level           │   0 │      0 │   1.846 │    1.232 │   0 │     1 │    2 │     3 │    4 │ ▅▇ ▇▅▃ │  │
│ │ curiosity_level           │   0 │      0 │   6.544 │    3.157 │   0 │     4 │    7 │    10 │   10 │ ▂▃▁▃▃▇ │  │
│ │ preferred_occupation3     │   0 │      0 │    2.79 │    1.693 │   0 │     2 │    3 │     4 │    6 │ ▃▅▇▆▆▅ │  │
│ │ preferred_occupation2     │   0 │      0 │   2.974 │    2.006 │   0 │     1 │    3 │     5 │    6 │ ▃▅▅▅▅▇ │  │
│ │ organized_chaos           │   0 │      0 │   4.206 │

In [ ]:
#saving the synthetic data to a CSV file
synthetic_data_normalized.to_csv('dating_dataset.csv', index=False)
print("Synthetic data has been saved to 'dating_dataset.csv'")

Synthetic data has been saved to 'dating_dataset.csv'
